In [10]:
import sys
import find_horizon_range_de
import find_horizon_range_network_de
import math
import pandas as pd


In [26]:
# Output schema (matches the docstring in find_horizon_range_de.py)
OUT_KEYS = [
    "range_mpc",
    "z_resp50",
    "z_resp10",         # z where detect_frac >= 0.1 (script variable is z_response90)
    "z_horizon",
    "vt_gpc3",          # constant comoving time-volume (Gpc^3) in the script output
    "z_det50",
    "z_det90",
    "z_sfr_det50",
    "z_sfr_det90",
    "z_mean",
    "z_sfr_mean",
]

def _as_dict(tup):
    if isinstance(tup, dict):
        return tup
    if not hasattr(tup, "__len__"):
        raise TypeError(f"Unexpected return type: {type(tup)}")
    if len(tup) != len(OUT_KEYS):
        raise ValueError(f"Expected {len(OUT_KEYS)} outputs, got {len(tup)}")
    return dict(zip(OUT_KEYS, tup))

def _fmt(x, nd=3):
    if x is None:
        return "None"
    if isinstance(x, (float, int)) and (math.isnan(x) or math.isinf(x)):
        return str(x)
    if isinstance(x, (float, int)):
        return f"{x:.{nd}f}"
    return str(x)

def print_result_block(label, d):
    # Print carefully with aligned keys
    print(f"\n=== {label} ===")
    print(f"Sky-averaged Range (Mpc)                  : {_fmt(d['range_mpc'], 1)}")
    print(f"Horizon distance (optimal orientation)    : z = {_fmt(d['z_horizon'])}")
    # print(f"Response z (50% detected)                 : {_fmt(d['z_resp50'])}")
    # print(f"Response z (10% detected)                 : {_fmt(d['z_resp10'])}")
    # print(f"Const. comoving time-volume (Gpc^3)       : {_fmt(d['vt_gpc3'])}")
    # print(f"z within which 50% of detected sources lie: {_fmt(d['z_det50'])}")
    # print(f"z within which 90% of detected sources lie: {_fmt(d['z_det90'])}")
    # print(f"SFR-weighted z50                          : {_fmt(d['z_sfr_det50'])}")
    # print(f"SFR-weighted z90                          : {_fmt(d['z_sfr_det90'])}")
    # print(f"Mean detected z                           : {_fmt(d['z_mean'])}")
    # print(f"SFR-weighted mean detected z              : {_fmt(d['z_sfr_mean'])}")

def results_to_df(results):
    # results: list of dicts each with label + OUT_KEYS
    return pd.DataFrame(results).set_index("label")

def run_network(label, m1, m2, dets, asd_array, pwfile_tag):
    res = []
    out = find_horizon_range_network_de.find_horizon_range(m1, m2, dets, asd_array, pwfile_tag)
    d = _as_dict(out)
    print_result_block(label, d)


In [28]:
# Network ASD list must match the detector order in NETWORK_DETS below.
ASD_NETWORK = {
    "HLVK_O5high": ["../../skymaps/psd/AplusDesign.txt", "../../skymaps/psd/AplusDesign.txt", "../../skymaps/psd/avirgo_O5high_NEW.txt", "../../skymaps/psd/kagra_128Mpc.txt"],
    "HLV_O5high": ["../../skymaps/psd/AplusDesign.txt", "../../skymaps/psd/AplusDesign.txt", "../../skymaps/psd/avirgo_O5high_NEW.txt",],
}

# Antenna power pattern file (pwfile) for the chosen network configuration
# (Must exist; generated / provided elsewhere in your pipeline.)
PWFILE = {
    "HLVK": "hlvk",
    "HLV": "hlv",
}

# Mass pairs
BNS = (1.4, 1.4)
NSBH = (1.4, 5.0)  # change BH mass if needed

# ------------------------------------------------------------
# Compute + print 
# ------------------------------------------------------------

HLVK_NETWORK = ["H", "L", "V", "J"] 

run_network(f"HLVK network O5 high — BNS {BNS}", *BNS, HLVK_NETWORK , ASD_NETWORK["HLVK_O5high"], PWFILE["HLVK"])
run_network(f"HLVK network O5 high — NSBH {NSBH})", *NSBH, HLVK_NETWORK , ASD_NETWORK["HLVK_O5high"], PWFILE["HLVK"])

# HLV_NETWORK = ["H", "L", "V"] 

# run_network("HLV network O5 high — BNS (1.4,1.4)", *BNS, HLV_NETWORK, ASD_NETWORK["HLV_O5high"], PWFILE["HLV"])
# run_network("HLV network O5 high — NSBH (1.4,5.0)", *NSBH, HLV_NETWORK, ASD_NETWORK["HLV_O5high"], PWFILE["HLV"])


horizon_redshift 0.16471005652517245

=== HLVK network O5 high — BNS (1.4, 1.4) ===
Sky-averaged Range (Mpc)                  : 387.1
Horizon distance (optimal orientation)    : z = 0.165
horizon_redshift 0.2614563640338607

=== HLVK network O5 high — NSBH (1.4, 5.0)) ===
Sky-averaged Range (Mpc)                  : 595.3
Horizon distance (optimal orientation)    : z = 0.261


In [ ]:
#This script generates pw files for GW network in the same format as provided template pw files.

import numpy as np
import lal

# --- detector mapping (match your conventions) ---
det_index = {
    'H': lal.LHO_4K_DETECTOR,
    'L': lal.LLO_4K_DETECTOR,
    'V': lal.VIRGO_DETECTOR,
    'J': lal.KAGRA_DETECTOR,          # KAGRA
}

def get_F(det, ra, dec, psi=0.0, time=0.0):
    gmst = lal.GreenwichMeanSiderealTime(lal.LIGOTimeGPS(time))
    resp = lal.CachedDetectors[det_index[det]].response
    return lal.ComputeDetAMResponse(resp, ra, dec, psi, gmst)

def network_rho2_geom(network, ra, dec, psi, iota, time=0.0):
    """
    Geometry-only network SNR^2 factor for non-precessing CBC:
      rho^2 ∝ Σ [F+^2 A+^2 + Fx^2 Ax^2]
    """
    cosi = np.cos(iota)
    Aplus  = 0.5 * (1.0 + cosi**2)
    Across = cosi
    s = 0.0
    for d in network:
        Fp, Fx = get_F(d, ra, dec, psi, time=time)
        s += (Fp*Fp)*(Aplus*Aplus) + (Fx*Fx)*(Across*Across)
    return s

def random_sky_orientation(N, rng):
    ra  = rng.uniform(0, 2*np.pi, N)
    u   = rng.uniform(-1, 1, N)
    dec = np.arcsin(u)                      # uniform on sky
    psi = rng.uniform(0, np.pi, N)          # polarization
    v   = rng.uniform(-1, 1, N)
    iota= np.arccos(v)                      # uniform in cos(iota)
    return ra, dec, psi, iota

def load_template_w_grid(template_pw_path):
    """
    Reads the first column of the template pw file and uses it verbatim as the w-grid.
    """
    w_template = np.loadtxt(template_pw_path, usecols=[0])
    if np.any(w_template <= 0) or np.any(w_template > 1.0 + 1e-9):
        raise ValueError("Template w-grid must be in (0,1].")
    # enforce ascending grid
    if not np.all(np.diff(w_template) > 0):
        raise ValueError("Template w-grid must be strictly increasing.")
    return w_template

def write_pw_like_template(network, out_pw_path, out_horizoncoord_path,
                           template_pw_path,
                           N_opt=200_000, N_mc=200_000,
                           seed=1234, time=0.0):
    """
    Generates:
      - horizon_coord_<tag>.txt (single row: ra dec psi iota)
      - pw_<tag>.txt in EXACT same line-grid/format as template pw file
    """
    rng = np.random.default_rng(seed)

    # --- 0) load template w-grid ---
    w_grid = load_template_w_grid(template_pw_path)

    # --- 1) find approx-optimal (ra,dec,psi) for face-on iota=0 by brute-force max ---
    ra_s, dec_s, psi_s, _ = random_sky_orientation(N_opt, rng)
    # vectorized-ish: still loop over N_opt because LAL call isn't vectorized
    rho2_s = np.empty(N_opt, dtype=float)
    for i in range(N_opt):
        rho2_s[i] = network_rho2_geom(network, ra_s[i], dec_s[i], psi_s[i], iota=0.0, time=time)

    imax = int(np.argmax(rho2_s))
    ra_opt, dec_opt, psi_opt = float(ra_s[imax]), float(dec_s[imax]), float(psi_s[imax])

    # save horizon coord (your pipeline expects ra dec psi iota)
    np.savetxt(
        out_horizoncoord_path,
        np.array([[ra_opt, dec_opt, psi_opt, 0.0]]),
        fmt="%.10e",
        header="ra dec psi iota"
    )

    rho2_opt = network_rho2_geom(network, ra_opt, dec_opt, psi_opt, iota=0.0, time=time)

    # --- 2) Monte Carlo for random sky+orientation ---
    ra_r, dec_r, psi_r, iota_r = random_sky_orientation(N_mc, rng)
    w = np.empty(N_mc, dtype=float)
    for i in range(N_mc):
        rho2 = network_rho2_geom(network, ra_r[i], dec_r[i], psi_r[i], iota_r[i], time=time)
        # projection factor in [0,1] by construction (numerical noise can push slightly >1; clip)
        w[i] = np.sqrt(max(rho2, 0.0) / rho2_opt)

    w = np.clip(w, 0.0, 1.0)
    w_sorted = np.sort(w)

    # --- 3) CCDF on the TEMPLATE grid: P(w >= w0) ---
    # Use "left" so that if w equals grid value, it counts as >=
    idx = np.searchsorted(w_sorted, w_grid, side="left")
    P = (len(w_sorted) - idx) / len(w_sorted)

    # Clean tiny negatives from floating error
    P = np.clip(P, 0.0, 1.0)

    # --- 4) write EXACT formatting: 4 decimals each, one space ---
    with open(out_pw_path, "w") as f:
        for wi, Pi in zip(w_grid, P):
            f.write(f"{wi:0.4f} {Pi:0.4f}\n")

    return (ra_opt, dec_opt, psi_opt), (w_grid[0], w_grid[-1], len(w_grid))


# -------------------------
# Example usage for HLVK
# -------------------------
network = ['H', 'L', 'V', 'K']  # HLVK
template_pw = "../data/pw_hlv.txt"  # your uploaded file

out_pw = "../data/pw_HLVK.txt"
out_hc = "../data/horizon_coord_HLVK.txt"

opt_angles, grid_info = write_pw_like_template(
    network=network,
    out_pw_path=out_pw,
    out_horizoncoord_path=out_hc,
    template_pw_path=template_pw,
    N_opt=200_000,
    N_mc=200_000,
    seed=1234,
)

print("Wrote:", out_pw)
print("Wrote:", out_hc)
print("Opt (ra, dec, psi):", opt_angles)
print("Template grid (w_min, w_max, n_lines):", grid_info)
